<a href="https://colab.research.google.com/github/aithaprasad/NLP_Kreyol_Segmentation/blob/main/Kreyol_With_HMM_Supervised.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import csv
with open('kreyol_segmentation_train.tsv', encoding="utf-8") as file:
  f = csv.reader(file, delimiter="\t")
  train_data = []
  for line in f:
    train_data.append(line)

In [2]:
train_data

[['konsiltan', 'k-on-s-i-l-t-an'],
 ['depotwa', 'd-e-p-o-t-w-a'],
 ['sosyopwofesyonèl', 's-o-s-y-o-p-w-o-f-e-s-y-o-n-è-l'],
 ['vejetal', 'v-e-j-e-t-a-l'],
 ['repibliye', 'r-e-p-i-b-l-i-y-e'],
 ['mason', 'm-a-s-on'],
 ['akt', 'a-k-t'],
 ['ewopeyèn', 'e-w-o-p-e-y-è-n'],
 ['Azèbayidjan', 'A-z-è-b-a-y-i-d-j-an'],
 ['teworis', 't-e-w-o-r-i-s'],
 ['dividal', 'd-i-v-i-d-a-l'],
 ['evite', 'e-v-i-t-e'],
 ['abòde', 'a-b-ò-d-e'],
 ['Iram', 'I-r-a-m'],
 ['pisans', 'p-i-s-an-s'],
 ['lèzòd', 'l-è-z-ò-d'],
 ['vantilatè', 'v-an-t-i-l-a-t-è'],
 ['bayè', 'b-a-y-è'],
 ['repondè', 'r-e-p-on-d-è'],
 ['pati', 'p-a-t-i'],
 ['tizonnen', 't-i-z-on-n-en'],
 ['jòf', 'j-ò-f'],
 ['teritwa', 't-e-r-i-t-w-a'],
 ['Touwon', 'T-ou-w-on'],
 ['lavil', 'l-a-v-i-l'],
 ['latant', 'l-a-t-an-t'],
 ['Tanis', 'T-a-n-i-s'],
 ['bawometrik', 'b-a-w-o-m-e-t-r-i-k'],
 ['voye', 'v-o-y-e'],
 ['epanouyi', 'e-p-a-n-ou-y-i'],
 ['konpayon', 'k-on-p-a-y-on'],
 ['dodomeya', 'd-o-d-o-m-e-y-a'],
 ['sèzyèm', 's-è-z-y-è-m'],
 ['chantè', 'ch-an-

In [3]:
res = []
for word_division in train_data:
  split_word = word_division[1].split('-')
  temp = []
  for letter in split_word:
    temp.append((letter[0], 'B'))
    if len(letter) > 1:
      for item in letter[1:]:
        temp.append((item, 'I'))
  res.append(temp)

In [4]:
res[0]

[('k', 'B'),
 ('o', 'B'),
 ('n', 'I'),
 ('s', 'B'),
 ('i', 'B'),
 ('l', 'B'),
 ('t', 'B'),
 ('a', 'B'),
 ('n', 'I')]

In [5]:
letter_tag_count = dict()
B_count, I_count = 0, 0 
unique_letters = []
for train in res:
  for letter_tag in train:
    letter, tag = letter_tag[0], letter_tag[1]
    
    if tag == 'B': B_count += 1
    else: I_count += 1

    if letter not in letter_tag_count:
      letter_tag_count[letter] = {tag : 1}
      unique_letters.append(letter)
    else:
      if tag not in letter_tag_count[letter]: letter_tag_count[letter][tag] = 1
      else: letter_tag_count[letter][tag] += 1

In [6]:
print(len(letter_tag_count))
print(len(unique_letters))
print(B_count)
print(I_count)

51
51
79696
7608


In [7]:
total_count_per_tag = {'B' : B_count, 'I': I_count}
for letter in unique_letters:
  new_dict = {'B' : 0, 'I': 0}
  tags = ['B', 'I']
  for tag in tags:
    if tag in letter_tag_count[letter]:
      new_dict[tag] = letter_tag_count[letter][tag] / total_count_per_tag[tag] 
  letter_tag_count[letter] = new_dict

In [13]:
word_context_tag_count = {'B' : {'B': 0, 'I': 0}, 'I' : {'B': 0, 'I': 0}}

for i in range(1, len(res)):
  for j in range(len(res[i])):
    if j == 0: continue
    letter, tag = res[i][j][0], res[i][j][1]
    prev_letter, prev_tag = res[i][j - 1][0], res[i][j - 1][1]
    word_context_tag_count[prev_tag][tag] += 1

In [14]:
word_context_tag_count

{'B': {'B': 61975, 'I': 7598}, 'I': {'B': 4903, 'I': 8}}

In [15]:
for tag in ['B', 'I']:
  total_tag_sum = sum(word_context_tag_count[tag].values())
  for next_tag in ['B', 'I']:
    word_context_tag_count[tag][next_tag] = (1 + word_context_tag_count[tag][next_tag]) / total_tag_sum

In [16]:
word_context_tag_count

{'B': {'B': 0.8908053411524586, 'I': 0.10922340563149498},
 'I': {'B': 0.9985746283852576, 'I': 0.0018326206475259622}}

In [17]:
def viterbi(sentence):
  state = []
  tags = ['B', 'I']
  for key, word in enumerate(sentence):
    p = []
    for tag in tags:
      emission_p = 0
      if key == 0: transmission_prob = word_context_tag_count['B'][tag]
      else: transmission_prob = word_context_tag_count[state[-1]][tag]
      if word in letter_tag_count.keys(): emission_p = letter_tag_count[word][tag]
  
      state_prop = emission_p * transmission_prob
      p.append(state_prop)
    max_value=max(p)
    value=tags[p.index(max_value)]
    state.append(value)
  return list(zip(sentence, state)), state

In [19]:
viterbi('dechouke')

([('d', 'B'),
  ('e', 'B'),
  ('c', 'B'),
  ('h', 'I'),
  ('o', 'B'),
  ('u', 'I'),
  ('k', 'B'),
  ('e', 'B')],
 ['B', 'B', 'B', 'I', 'B', 'I', 'B', 'B'])

In [ ]:
def predict_from_scratch(sentence_tag):
  
  correct = 0
  sentence, tags = [], []
  
  for word, tag in sentence_tag:
    sentence.append(word)
    tags.append(tag)
  
  pred_labels = viterbi(sentence)
  for i in range(len(tags)):
    if pred_labels[1][i] == tags[i]:
      correct += 1
  
  return correct


In [ ]:
def evaluate():
  total = 0
  correct_from_scratch = 0
  testfile = open('test.tsv', 'r')
  sentence = []
  for line in testfile:
    total += 1
    pieces = line.rstrip("\n").split("\t") 
    if pieces[0]=='<S>':
      correct_from_scratch += predict_from_scratch(sentence)
      sentence = []
    else:
      sentence.append(pieces)
  correct_from_scratch += predict_from_scratch(sentence)
  return correct_from_scratch/total